<a href="https://colab.research.google.com/github/DavorJordacevic/TensorFlow-Keras-Neural-Networks/blob/master/TensorFlow_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O cats_and_dogs_filtered.zip

In [0]:
with zipfile.ZipFile("cats_and_dogs_filtered.zip","r") as zip_ref:
    zip_ref.extractall()

In [0]:
base_dir = 'cats_and_dogs_filtered'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [0]:
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2

In [0]:
base_model = MobileNetV2(input_shape=(224, 224, 3),
                                      include_top=False,
                                      weights='imagenet')

In [0]:
base_model.trainable = False

In [0]:
model = Sequential([base_model,
                    GlobalAveragePooling2D(),
                    Dense(1, activation='sigmoid')])

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [0]:
history = model.fit(
      train_generator,
      epochs=6,
      validation_data=validation_generator,
      verbose=2)

In [0]:
# Converting a tf.Keras model to a TensorFlow Lite model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

8861428

In [0]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

In [0]:
# Get input and output tensors.

input_details = interpreter.get_input_details()
output_details= interpreter.get_output_details()

print("== Input details ==")
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])
print("\n== Output details ==")
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
shape: [  1 224 224   3]
type: <class 'numpy.float32'>

== Output details ==
shape: [1 1]
type: <class 'numpy.float32'>


In [0]:
interpreter.resize_tensor_input(input_details[0]['index'], (32, 224, 224, 3))
interpreter.resize_tensor_input(output_details[0]['index'], (32, 5))
interpreter.allocate_tensors()

In [0]:
# Get input and output tensors.

input_details = interpreter.get_input_details()
output_details= interpreter.get_output_details()

print("== Input details ==")
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])
print("\n== Output details ==")
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
shape: [ 32 224 224   3]
type: <class 'numpy.float32'>

== Output details ==
shape: [32  1]
type: <class 'numpy.float32'>


In [0]:
image_batch, label_batch = next(iter(validation_generator))

In [0]:
interpreter.set_tensor(input_details[0]['index'], image_batch)

In [0]:
interpreter.invoke()

In [0]:
tflite_results = interpreter.get_tensor(output_details[0]['index'])

tf_results = model.predict(image_batch)

for tf_result, tflite_result in zip(tf_results, tflite_results):
  np.testing.assert_almost_equal(tf_result, tflite_result, decimal=5)

In [0]:
def predict_classes(tf_model, tf_lite_interpreter, x):
  tf_lite_interpreter.set_tensor(input_details[0]['index'], x)
  tf_lite_interpreter.invoke()
  tflite_results = tf_lite_interpreter.get_tensor(output_details[0]['index'])

  tf_results = model.predict_classes(x)

  if tflite_results.shape[-1] > 1:
    tflite_results = tflite_results.argmax(axis=-1)
  else:
    tflite_results = (tflite_results > 0.5).astype('int32')
    
  shape = (tf_results == tflite_results).shape

  return (tf_results == tflite_results).reshape((shape[1], shape[0]))

In [0]:
print((predict_classes(model, interpreter, image_batch)))

[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True]]


In [0]:
############################################################

In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_quant_model)

2299040

In [0]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_quant_model)
interpreter.allocate_tensors()

In [0]:
# Get input and output tensors.

input_details = interpreter.get_input_details()
output_details= interpreter.get_output_details()

interpreter.resize_tensor_input(input_details[0]['index'], (32, 224, 224, 3))
interpreter.resize_tensor_input(output_details[0]['index'], (32, 5))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details= interpreter.get_output_details()

In [0]:
print((predict_classes(model, interpreter, image_batch)))

[[False  True  True  True  True  True False  True  True  True False  True
  False False  True False  True  True  True  True  True False  True  True
   True  True  True  True  True False  True False]]


In [0]:
def eval_model(interpreter, validation_generator):
  total_seen = 0
  num_correct = 0

  for img, label in validation_generator:
    total_seen += img.shape[0]

    interpreter.set_tensor(input_details[0]['index'], img)
    interpreter.invoke()
    predictions = interpreter.get_tensor(output_details[0]['index'])

    if predictions.shape[-1] > 1:
      predictions = predictions.argmax(axis=-1)
    else:
      predictions = (predictions > 0.5).astype('int32')

    num_correct += np.sum(predictions == label.reshape(predictions.shape))

    if total_seen % 10 == 0:
      print("Accuracy after %i images: %f" %
            (total_seen, float(num_correct) / float(total_seen)))

  return float(num_correct) / float(total_seen)

In [0]:
print(eval_model(interpreter, validation_generator))



---

